In [1]:
DATABASE='googol'
HOST='127.0.0.1'
USERNAME='username'
PASSWORD='password'
PORT='5432'

In [2]:
import torch
import torch.nn.functional as F
from sentence_transformers import SentenceTransformer
from transformers.dynamic_module_utils import get_imports
from unittest.mock import patch
from numpy.linalg import norm
import numpy as np
import psycopg2

/Users/kenjiwang/anaconda3/envs/search-engine/lib/python3.12/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [3]:
conn = psycopg2.connect(database=DATABASE,
	host=HOST,
	user=USERNAME,
	password=PASSWORD,
	port=PORT
)

cursor = conn.cursor()

In [4]:
model = SentenceTransformer('dunzhang/stella_en_1.5B_v5', trust_remote_code=True)

In [5]:
def search_googol(query):
	query_formatted = query.replace('\'', '\'\'')
	query_emb = model.encode([query], normalize_embeddings=True)[0].tolist()

	pg_query = f"SELECT url, title, description FROM websites ORDER BY description_emb <=> '{query_emb}', title_emb <=> '{query_emb}' LIMIT 10;"

	cursor.execute(pg_query)
	many_rows = cursor.fetchmany(10)

	for row in many_rows:
		print(row)

In [6]:
search_googol("search engine")

('google.com', 'Google', "Search the world's information, including webpages, images, videos and more. Google has many special features to help you find exactly what you're looking for.")
('twitter.com', 'x.com', '')
('googletagmanager.com', 'Error 404 (Not Found)!!1', '')
('facebook.com', 'Facebook - log in or sign up', 'Log into Facebook to start sharing and connecting with your friends, family, and people you know.')
('youtube.com', 'YouTube', 'Enjoy the videos and music you love, upload original content, and share it all with friends, family, and the world on YouTube.')
